In [1]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END

## Testing execution plan conversion


In [ ]:
from rich import print as rprint

rprint(simple_graph._convert_execution_plan())

## Testing execution


In [16]:
import time
from datetime import datetime

egraph = Graph()


@egraph.node()
def a():
    time.sleep(1)
    print("a \n", datetime.now())


@egraph.node()
def b():
    time.sleep(1)
    print("b \n", datetime.now())


@egraph.node()
def c():
    time.sleep(1)
    print("c \n", datetime.now())


@egraph.node()
def d():
    time.sleep(1)
    print("d \n", datetime.now())


@egraph.node()
def e():
    time.sleep(1)
    print("e \n", datetime.now())


@egraph.node()
def f():
    time.sleep(1)
    print("f", datetime.now())


@egraph.node()
def g():
    time.sleep(1)
    print("g \n", datetime.now())


egraph.add_edge(START, "a")
egraph.add_edge("a", "b")
egraph.add_edge("b", "c")
egraph.add_edge("b", "d")
egraph.add_edge("d", "e")
egraph.add_edge("c", "f")
egraph.add_edge("f", "g")
egraph.add_edge("e", "g")
egraph.add_edge("g", END)

egraph.compile()
egraph.visualize()

In [ ]:
egraph.execute()

In [10]:
egraph = Graph()


@egraph.node()
def a():
    time.sleep(1)
    print("a", datetime.now())


@egraph.node()
def b():
    time.sleep(1)
    print("b", datetime.now())


@egraph.node()
def c():
    time.sleep(1)
    print("c", datetime.now())


@egraph.node()
def d():
    time.sleep(1)
    print("d", datetime.now())


@egraph.node()
def e():
    time.sleep(1)
    print("e", datetime.now())


@egraph.node()
def f():
    time.sleep(1)
    print("f", datetime.now())


@egraph.node()
def g():
    time.sleep(1)
    time.sleep(1)
    print("g", datetime.now())


egraph.add_edge(START, "a")
egraph.add_edge("a", "b")
egraph.add_edge("b", "c")
egraph.add_edge("c", "d")
egraph.add_edge("d", "e")
egraph.add_edge("e", "f")
egraph.add_edge("f", "g")
egraph.add_edge("g", END)

egraph.compile()
egraph.visualize()

In [ ]:
egraph.execute()

In [ ]:
from rich import print as rprint

# Create a test graph with a mix of parallel and sequential paths
test_graph = Graph()


@test_graph.node()
def start_task():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting workflow \n")
    time.sleep(1)


@test_graph.node()
def parallel_task_1():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting parallel task 1 \n")
    time.sleep(4)
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Finished parallel task 1 \n")


@test_graph.node()
def parallel_task_2():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting parallel task 2 \n")
    time.sleep(4)
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Finished parallel task 2 \n")


@test_graph.node()
def final_task():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Running final task \n")
    time.sleep(1)


# Create a workflow with parallel execution
test_graph.add_edge(START, "start_task")
test_graph.add_edge("start_task", "parallel_task_1")
test_graph.add_edge("start_task", "parallel_task_2")
test_graph.add_edge("parallel_task_1", "final_task")
test_graph.add_edge("parallel_task_2", "final_task")
test_graph.add_edge("final_task", END)

# Compile and execute
test_graph.compile()
print("\nExecution Plan:")
rprint(test_graph.execution_plan)
print("\nStarting execution:")
start_time = time.time()
test_graph.execute()
end_time = time.time()
print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

## Testing state


#### Test 1


In [ ]:
from tiny_graph.models.base import GraphState
from typing import Dict
import time
from tiny_graph.buffer.factory import History, Incremental, LastValue


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Test Incremental Buffer
@test_graph.node()
def add_to_counter(state):
    time.sleep(0.5)
    return {"counter": 5}


@test_graph.node()
def add_more_to_counter(state):
    time.sleep(0.5)
    return {"counter": 3}


# Test History Buffer
@test_graph.node()
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.95, "loss": 0.1}}


@test_graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.88}}


# Test LastValue Buffer
@test_graph.node()
def set_status_running(state):
    time.sleep(0.5)
    return {"current_status": "running"}


@test_graph.node()
def set_status_complete(state):
    time.sleep(0.5)
    return {"current_status": "complete"}


# Create the workflow
test_graph.add_edge(START, "add_to_counter")
test_graph.add_edge("add_to_counter", "add_more_to_counter")
test_graph.add_edge("add_more_to_counter", "add_metrics")
test_graph.add_edge("add_metrics", "update_metrics")
test_graph.add_edge("update_metrics", "set_status_running")
test_graph.add_edge("set_status_running", "set_status_complete")
test_graph.add_edge("set_status_complete", END)

# Compile and execute
test_graph.compile()
test_graph.execute(timeout=10)

# Print final state
print("\nFinal State:")
print(f"Counter: {test_graph.state.counter}")  # Should be 8 (5 + 3)
print(
    f"Metrics History: {test_graph.state.metrics}"
)  # Should contain both metric updates
print(f"Current Status: {test_graph.state.current_status}")  # Should be "complete"

In [ ]:
test_graph.visualize()

#### Test 2


In [ ]:
from tiny_graph.models.base import GraphState
from typing import Dict
import time
from tiny_graph.buffer.factory import History, Incremental, LastValue
from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@test_graph.node()
def increment_counter(state):
    time.sleep(0.5)
    return {"counter": 1}


@test_graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    return {"counter": -1}


@test_graph.node()
def update_status(state):
    time.sleep(0.5)
    return {"current_status": "in_progress"}


# Create the workflow with parallel execution
test_graph.add_edge(START, "increment_counter")
test_graph.add_edge(START, "decrement_counter")
test_graph.add_edge(START, "update_status")
test_graph.add_edge("increment_counter", END)
test_graph.add_edge("decrement_counter", END)
test_graph.add_edge("update_status", END)

# Compile and execute
test_graph.compile()
test_graph.execute(timeout=10)

# Print final state
print("\nFinal State:")
print(
    f"Counter: {test_graph.state.counter}"
)  # Should reflect the net effect of increments and decrements
print(
    f"Metrics History: {test_graph.state.metrics}"
)  # Should be empty as no metrics are updated
print(f"Current Status: {test_graph.state.current_status}")  # Should be "in_progress"

In [ ]:
test_graph.execution_plan

In [ ]:
test_graph.visualize()

In [ ]:
graph = Graph()


@graph.node()
def escape():
    print("Starting workflow")


@graph.node()
def process_data():
    print("Processing data")


@graph.node()
def validate():
    print("Validating results")


@graph.node()
def aa():
    print("Validating results")


@graph.node()
def bb():
    print("Validating results")


@graph.node()
def prep():
    print("Workflow complete")


graph.add_edge(START, "process_data")
graph.add_edge("process_data", "validate")
graph.add_edge("validate", "escape")
graph.add_edge("escape", "prep")
graph.add_edge("validate", "aa")
graph.add_edge("aa", "bb")
graph.add_edge("bb", "prep")
graph.add_edge("prep", END)

graph.compile()
graph.visualize()

In [ ]:
graph.execution_plan

#### Test 3


In [ ]:
import time
from tiny_graph.graph.executable import Graph
from tiny_graph.models.base import GraphState
from tiny_graph.buffer.factory import Incremental, LastValue, History
from typing import Dict
from rich import print as rprint


# Define a state model with buffer types
class ComplexTestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    status: LastValue[str]  # Will only keep last value
    metrics: History[Dict[str, float]]  # Will keep history of all updates


# Initialize the graph with state
state = ComplexTestState(counter=0, status="", metrics={})
graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@graph.node()
def increment_counter(state):
    time.sleep(0.5)
    return {"counter": 2}


@graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    return {"counter": -1}


@graph.node()
def update_status_to_in_progress(state):
    time.sleep(0.5)
    return {"status": "in_progress"}


@graph.node()
def update_status_to_complete(state):
    time.sleep(0.5)
    return {"status": "complete"}


@graph.node()
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.9, "loss": 0.1}}


@graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.85}}


@graph.node()
def finalize_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.01, "precision": 0.99}}


# Create the workflow with multiple levels of execution
graph.add_edge(START, "increment_counter")
graph.add_edge(START, "decrement_counter")
graph.add_edge(START, "update_status_to_in_progress")
graph.add_edge("increment_counter", "add_metrics")
graph.add_edge("decrement_counter", "add_metrics")
graph.add_edge("add_metrics", "update_metrics")
graph.add_edge("update_metrics", "finalize_metrics")
graph.add_edge("update_status_to_in_progress", "update_status_to_complete")
graph.add_edge("update_status_to_complete", "finalize_metrics")
graph.add_edge("finalize_metrics", END)

# Compile and execute
graph.compile()


graph.execute(timeout=10)

rprint(graph.state)

# Assert final state
assert (
    graph.state.counter == 1
), "Counter should reflect net effect of increments and decrements"
assert graph.state.status == "complete", "Status should be 'complete'"
assert len(graph.state.metrics) == 3, "Metrics should contain three updates"

In [ ]:
graph.visualize()

#### Test 4


In [ ]:
simple_graph = Graph()


# Define some example actions
@simple_graph.node(interrupt="before")
def escape():
    print("Starting workflow")


@simple_graph.node()
def process_data():
    print("Processing data")


@simple_graph.node()
def validate():
    print("Validating results")


@simple_graph.node()
def aa():
    print("Validating results")


@simple_graph.node()
def bb():
    print("Validating results")


@simple_graph.node()
def dd():
    print("Validating results")


@simple_graph.node()
def cc():
    print("Validating results")


@simple_graph.node()
def hh():
    print("Validating results")


@simple_graph.node()
def prep():
    print("Workflow complete")


# Add edges to create workflow
simple_graph.add_edge(START, "process_data")
simple_graph.add_edge("process_data", "validate")
simple_graph.add_edge("validate", "escape")
simple_graph.add_edge("escape", "dd")
simple_graph.add_edge("escape", "cc")
simple_graph.add_edge("cc", "hh")
simple_graph.add_edge("dd", "hh")
simple_graph.add_edge("hh", "prep")
simple_graph.add_edge("validate", "aa")
simple_graph.add_edge("aa", "bb")
simple_graph.add_edge("bb", "prep")
simple_graph.add_edge("prep", END)

simple_graph.compile()

In [ ]:
simple_graph.visualize()

In [ ]:
simple_graph.execution_plan

In [ ]:
from rich import print as rprint

rprint(simple_graph.execution_plan)

In [ ]:
rprint(simple_graph._convert_execution_plan())

In [ ]:
def slice_execution_plan(plan, metadata):
    def slice_list(lst, meta):
        result = []
        for item in lst:
            if isinstance(item, list):
                # Recursively slice nested lists
                sliced_item = slice_list(item, meta)
                if sliced_item:
                    result.append(sliced_item)
            else:
                # Check metadata for slicing
                if meta.get(item) == "before":
                    break
                result.append(item)
                if meta.get(item) == "after":
                    break
        return result

    return slice_list(plan, metadata)


# Example usage
execution_plan = [
    "process_data",
    "validate",
    [["escape", ["dd", "cc"], "hh"], ["aa", "bb"]],
    "prep",
]
metadata = {
    # 'process_data': 'after',
    # 'validate': 'before',
    # 'escape': 'after',
    "dd": "before",
    # 'cc': 'after',
    # 'hh': 'before',
    # 'aa': 'after',
    # 'bb': 'before',
    # 'prep': 'after'
}

sliced_plan = slice_execution_plan(execution_plan, metadata)
print(sliced_plan)

In [ ]:
execution_plan[2:]

In [ ]:
simple_graph.execute()

In [ ]:
# problem here is that in the case of update_status_to_complete,
# it's not checking if the next node is a convergence point

# it needs to have a function that will check if the next node
# connector. find_convergence_point is not doing enough.

## Testing interruption


### Test 1


In [2]:
from tiny_graph.models.base import GraphState
from typing import Dict
import time
from tiny_graph.buffer.factory import History, Incremental, LastValue


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Test Incremental Buffer
@test_graph.node()
def add_to_counter(state):
    time.sleep(0.5)
    return {"counter": 5}


@test_graph.node()
def add_more_to_counter(state):
    time.sleep(0.5)
    return {"counter": 3}


# Test History Buffer
@test_graph.node(interrupt="before")
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.95, "loss": 0.1}}


@test_graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.88}}


# Test LastValue Buffer
@test_graph.node()
def set_status_running(state):
    time.sleep(0.5)
    return {"current_status": "running"}


@test_graph.node()
def set_status_complete(state):
    time.sleep(0.5)
    return {"current_status": "complete"}


# Create the workflow
test_graph.add_edge(START, "add_to_counter")
test_graph.add_edge("add_to_counter", "add_more_to_counter")
test_graph.add_edge("add_more_to_counter", "add_metrics")
test_graph.add_edge("add_metrics", "update_metrics")
test_graph.add_edge("update_metrics", "set_status_running")
test_graph.add_edge("set_status_running", "set_status_complete")
test_graph.add_edge("set_status_complete", END)

# Compile and execute
test_graph.compile()
test_graph.visualize()

DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.viewing.view(['quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.quote(['is_html_string', 'is_valid_id', 'dot_keywords', 'endswith_odd_number_of_backslashes', 'escape_unescaped_quotes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.a_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.attr_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.dot.Dot.clear(['keep_attrs'])
DEBUG:graphviz._tools:deprecate posit

In [3]:
test_graph.execute()

DEBUG:tiny_graph.graph.executable:Executing node: add_to_counter
DEBUG:tiny_graph.graph.executable:Executing task in node: add_to_counter
DEBUG:tiny_graph.graph.executable:State updated after node: add_to_counter
DEBUG:tiny_graph.graph.executable:Executing node: add_more_to_counter
DEBUG:tiny_graph.graph.executable:Executing task in node: add_more_to_counter
DEBUG:tiny_graph.graph.executable:State updated after node: add_more_to_counter
DEBUG:tiny_graph.graph.executable:State updated after node: add_metrics
DEBUG:tiny_graph.graph.executable:State updated after node: add_metrics
INFO:tiny_graph.graph.executable:Chain paused.


In [4]:
test_graph.execution_plan

[ExecutableNode(node_name='add_to_counter', task_list=[<function add_to_counter at 0x10c8365c0>], execution_type='sequential', interrupt=None),
 ExecutableNode(node_name='add_more_to_counter', task_list=[<function add_more_to_counter at 0x10c8359e0>], execution_type='sequential', interrupt=None),
 ExecutableNode(node_name='add_metrics', task_list=[<function add_metrics at 0x10c836480>], execution_type='sequential', interrupt='before'),
 ExecutableNode(node_name='update_metrics', task_list=[<function update_metrics at 0x10563bce0>], execution_type='sequential', interrupt=None),
 ExecutableNode(node_name='set_status_running', task_list=[<function set_status_running at 0x10c836700>], execution_type='sequential', interrupt=None),
 ExecutableNode(node_name='set_status_complete', task_list=[<function set_status_complete at 0x10c836660>], execution_type='sequential', interrupt=None)]

In [5]:
test_graph.next_execution_node

'add_metrics'

In [6]:
test_graph.resume()

DEBUG:tiny_graph.graph.executable:State updated after node: add_metrics
DEBUG:tiny_graph.graph.executable:State updated after node: add_metrics
INFO:tiny_graph.graph.executable:Chain paused.


In [7]:
from rich import print as rprint

rprint(test_graph.execution_plan)

[
    ExecutableNode(
        node_name='add_to_counter',
        task_list=[<function add_to_counter at 0x10c8365c0>],
        execution_type='sequential',
        interrupt=None
    ),
    ExecutableNode(
        node_name='add_more_to_counter',
        task_list=[<function add_more_to_counter at 0x10c8359e0>],
        execution_type='sequential',
        interrupt=None
    ),
    ExecutableNode(
        node_name='add_metrics',
        task_list=[<function add_metrics at 0x10c836480>],
        execution_type='sequential',
        interrupt='before'
    ),
    ExecutableNode(
        node_name='update_metrics',
        task_list=[<function update_metrics at 0x10563bce0>],
        execution_type='sequential',
        interrupt=None
    ),
    ExecutableNode(
        node_name='set_status_running',
        task_list=[<function set_status_running at 0x10c836700>],
        execution_type='sequential',
        interrupt=None
    ),
    ExecutableNode(
        node_name='set_status_complete',
        task_list=[<function set_status_complete at 0x10c836660>],
        execution_type='sequential',
        interrupt=None
    )
]